In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [4]:
df = pd.read_csv('/content/CC GENERAL.csv')

In [5]:
df.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [6]:
df = df.drop('CUST_ID', axis=1)
df.head()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [7]:
df.shape

(8950, 17)

In [8]:
df.isnull().sum()

BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64

In [9]:
df['CREDIT_LIMIT'].min()

50.0

In [10]:
df['CREDIT_LIMIT'] = df['CREDIT_LIMIT'].fillna(50.0)

In [11]:
df['MINIMUM_PAYMENTS'].min()

0.019163

In [12]:
df['MINIMUM_PAYMENTS'] = df['MINIMUM_PAYMENTS'].fillna(0.019163)

In [13]:
df = df[df['BALANCE'] < df['BALANCE'].quantile(.95)]
df = df[df['PURCHASES'] < df['PURCHASES'].quantile(.95)]
df = df[df['MINIMUM_PAYMENTS'] < df['MINIMUM_PAYMENTS'].quantile(.95)]
df = df[df['PAYMENTS'] < df['PAYMENTS'].quantile(.95)]
df = df[df['CREDIT_LIMIT'] < df['CREDIT_LIMIT'].quantile(.95)]
df = df[df['PURCHASES_TRX'] < df['PURCHASES_TRX'].quantile(.95)]
df = df[df['ONEOFF_PURCHASES'] < df['ONEOFF_PURCHASES'].quantile(.95)]
df = df[df['CASH_ADVANCE'] < df['CASH_ADVANCE'].quantile(.95)]
df = df[df['CASH_ADVANCE_TRX'] < df['CASH_ADVANCE_TRX'].quantile(.95)]

In [14]:
df.shape

(5550, 17)

In [15]:
df.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000,5550.000000
mean,832.486080,0.841700,432.484301,188.472960,244.347683,363.703221,0.449513,0.127538,0.339292,0.081100,1.340721,7.907027,3029.292538,784.546855,348.946341,0.155732,11.448468
std,1027.513373,0.265193,516.419191,319.834004,396.293021,658.720724,0.385939,0.225164,0.381710,0.125014,2.144479,8.390110,2082.776936,745.763625,355.879921,0.292852,1.435887
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.019163,0.000000,6.000000
25%,53.400141,0.756945,32.622500,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,1.000000,1500.000000,284.394602,134.213938,0.000000,12.000000
50%,412.841852,1.000000,259.300000,0.000000,70.685000,0.000000,0.416667,0.000000,0.166667,0.000000,0.000000,6.000000,2500.000000,552.815654,198.379064,0.000000,12.000000
75%,1268.013678,1.000000,633.830000,250.000000,343.947500,436.852575,0.833333,0.166667,0.666667,0.142857,2.000000,12.000000,4000.000000,1033.232117,454.651688,0.166667,12.000000
max,5901.592192,1.000000,3654.080000,1370.000000,3362.670000,3058.096526,1.000000,1.000000,1.000000,0.900000,9.000000,38.000000,9100.000000,4256.495061,1919.690041,1.000000,12.000000


In [16]:
scaler = StandardScaler()

In [17]:
normalized_df = scaler.fit_transform(df)

In [ ]:
import matplotlib.pyplot as plt

alvo = df
max = 15
km_silhouette = []
for i in range(2,max):
  km = KMeans(n_clusters = i, random_state=0).fit(alvo)
  preds = km.predict(alvo)
  km_silhouette.append(km.score(alvo))
  silhouette = silhouette_score(alvo,preds)
  km_silhouette.append(silhouette)
plt.figure(figsize=(7,4))
plt.title("Silhouette Method", fontsize=16)
plt.scatter(x=[i for i in range(2,max)],y=km_silhouette,s=150,edgecolor='k')
plt.grid(True)
plt.xlabel("Nº de clusters", fontsize=14)
plt.ylabel("Silhouette score", fontsize=15)
plt.xticks([i for i in range(2,max)], fontsize=14)
plt.yticks(fontsize=15)
plt.show()